<a href="https://colab.research.google.com/github/MauricioCastroF/NLP-desafios/blob/main/Desaf%C3%ADo_2_NLP_CastroFMauricio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

# ***Dataset seleccionado***: IMDB dataset. Corresponde a comentarios/opiniones sobre el contenido de películas, recopilados por la plataforma IMDB

In [2]:
df = pd.read_csv('/content/sample_data/IMDB Dataset.csv', index_col = False, sep='/n')
df.head()

<ipython-input-2-512bcc6a53e1>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/sample_data/IMDB Dataset.csv', index_col = False, sep='/n')
<ipython-input-2-512bcc6a53e1>:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv('/content/sample_data/IMDB Dataset.csv', index_col = False, sep='/n')


,"review,sentiment;;;"
0,One of the other reviewers has mentioned that ...
1,"""A wonderful little production. <br /><br />Th..."
2,"""I thought this was a wonderful way to spend t..."
3,Basically there's a family where a little boy ...
4,"""Petter Mattei's """"Love in the Time of Money""""..."


In [3]:
print("La cantidad de documentos es:", df.shape[0])

La cantidad de documentos es: 18498


1. ***Preprocesamiento***: Se procede a identificar/separar oraciones y palabras

In [4]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

2. Crear los vectores (word2vec)

In [5]:
from gensim.models.callbacks import CallbackAny2Vec
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [6]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=50,       # dimensionalidad de los vectores
                     negative=30,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram

In [7]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [8]:
# Cantidad de filas/docs encontradas en el corpus
print("La cantidad de docs en el corpus es:", w2v_model.corpus_count)

La cantidad de docs en el corpus es: 18498


In [9]:
# Cantidad de words encontradas en el corpus
print("La cantidad de words distintas en el corpus es:", len(w2v_model.wv.index_to_key))

La cantidad de words distintas en el corpus es: 26238


3. ***Entrenar el modelo generador***

In [10]:
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 10661300.0
Loss after epoch 1: 8657198.0
Loss after epoch 2: 8384856.0
Loss after epoch 3: 8249986.0
Loss after epoch 4: 8256496.0
Loss after epoch 5: 8118620.0
Loss after epoch 6: 8011780.0
Loss after epoch 7: 7184740.0
Loss after epoch 8: 2811648.0
Loss after epoch 9: 2777696.0
Loss after epoch 10: 2744056.0
Loss after epoch 11: 2702288.0
Loss after epoch 12: 2667504.0
Loss after epoch 13: 2622920.0
Loss after epoch 14: 2579096.0
Loss after epoch 15: 2529080.0
Loss after epoch 16: 2482336.0
Loss after epoch 17: 2426848.0
Loss after epoch 18: 2368256.0
Loss after epoch 19: 2305208.0


(64047582, 87140880)

4. ***Ensayar***

Palabras que más se relacionan:

In [11]:
w2v_model.wv.most_similar(positive=["torture"], topn=20)

[('duckling', 0.6233449578285217),
 ('pretend', 0.5832557678222656),
 ('defeat', 0.5708845853805542),
 ('nuke', 0.5499873757362366),
 ('bore', 0.5430389642715454),
 ('boil', 0.5429417490959167),
 ('vomit', 0.5410196185112),
 ('blink', 0.5298225283622742),
 ('drink', 0.5295728445053101),
 ('disgusting', 0.5287187695503235),
 ('cause', 0.5152779221534729),
 ('carnage', 0.5097328424453735),
 ('shoot', 0.5085132718086243),
 ('wash', 0.5074636936187744),
 ('rape', 0.5074612498283386),
 ('escape', 0.5027670860290527),
 ('apply', 0.49643421173095703),
 ('scream', 0.4936883747577667),
 ('bloody', 0.48984774947166443),
 ('satisfy', 0.4860241115093231)]

In [12]:
w2v_model.wv.most_similar(positive=["summer"], topn=10)

[('quarter', 0.6597265005111694),
 ('weekend', 0.6477310061454773),
 ('month', 0.6346899271011353),
 ('reunion', 0.6310725808143616),
 ('night', 0.6238582134246826),
 ('evening', 0.615725040435791),
 ("year's", 0.6045517325401306),
 ('holiday', 0.6033679246902466),
 ('matinée', 0.5990145802497864),
 ('august', 0.5925398468971252)]

Palabras que menos se relacionan:

In [13]:
w2v_model.wv.most_similar(negative=["wonderful"], topn=20)

[('housewives', 0.538569986820221),
 ('somethings', 0.4628709852695465),
 ('wingers', 0.4468528628349304),
 ('ft', 0.4383505582809448),
 ('conveniently', 0.4375515282154083),
 ("tenant's", 0.43575578927993774),
 ('toxie', 0.4299646317958832),
 ('pound', 0.42747893929481506),
 ('yards', 0.42588141560554504),
 ('carrots', 0.4233446717262268),
 ('iameracing', 0.4186707139015198),
 ('boomers', 0.41806674003601074),
 ('towers', 0.41714543104171753),
 ('enemy', 0.41708263754844666),
 ('rabbits', 0.41640210151672363),
 ('jewelry', 0.4143434762954712),
 ('meters', 0.40894678235054016),
 ('clerk', 0.408935546875),
 ('suns', 0.4007912874221802),
 ('guards', 0.4001000225543976)]

In [14]:
w2v_model.wv.most_similar(negative=["shaw"], topn=20)

[('seriously', 0.4967474043369293),
 ('damn', 0.4900675117969513),
 ("that's", 0.45415621995925903),
 ('just', 0.4128207266330719),
 ('basically', 0.40576228499412537),
 ('itself', 0.4033470153808594),
 ('feels', 0.4012058973312378),
 ("what's", 0.38773977756500244),
 ('approach', 0.386709600687027),
 ("it's", 0.38576018810272217),
 ('whole', 0.36391422152519226),
 ('simple', 0.36316418647766113),
 ('sounds', 0.3576790690422058),
 ('ultimately', 0.3518756330013275),
 ('seems', 0.3514268398284912),
 ('nearly', 0.34503257274627686),
 ('depressing', 0.3430941104888916),
 ('anyway', 0.3426712155342102),
 ('easy', 0.34109199047088623),
 ('again', 0.33731013536453247)]

5. Visualizar agrupación de vectores

In [15]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model):
    num_dimensions = 2

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [16]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=50
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

Conclusiones:
* 18 mil comentarios con 26 palabras diferentes, relacionadas con opiniones sobre películas fueron analizados mediante una técnica de Word Embedding
* Las relaciones de interés que más me llamaron la atención están entre las palabras wonderful y convenient, o carrot, no la esperaba. En la figura 2D, no logro identificar relaciones entre palabras que me orienten a sacar conclusiones robustas o pertinentes. Es posible que tenga que mejorar el entrenamiento del modelo